In [72]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import json
from unidecode import unidecode
from tqdm.contrib.concurrent import thread_map
from functools import partial

In [3]:
feeds_url = 'https://www.demorgen.be/sitemaps/news.xml'
page_divs = requests.get(feeds_url).content
soup = BeautifulSoup(page_divs, "html.parser")
print(soup.prettify())

<?xml version="1.0" encoding="UTF-8"?>
<urlset xmlns="http://www.sitemaps.org/schemas/sitemap/0.9" xmlns:news="http://www.google.com/schemas/sitemap-news/0.9">
 <url>
  <loc>
   https://www.demorgen.be/snelnieuws/chinese-zoo-in-de-verdediging-hun-honingberen-zijn-echt-en-geen-personeel-in-een-berenpak~b84dab72/
  </loc>
  <lastmod>
   2023-08-02T14:07:09.895Z
  </lastmod>
  <news:news>
   <news:publication>
    <news:name>
     De Morgen
    </news:name>
    <news:language>
     nl
    </news:language>
   </news:publication>
   <news:publication_date>
    2023-08-02T13:56:17.353Z
   </news:publication_date>
   <news:title>
    ▶ Chinese zoo in de verdediging: hun honingberen zijn écht, en geen personeel in een berenpak
   </news:title>
  </news:news>
 </url>
 <url>
  <loc>
   https://www.demorgen.be/meningen/zak-cartoon-van-de-dag~b0de26ebb/
  </loc>
  <lastmod>
   2023-08-02T13:52:21.127Z
  </lastmod>
  <news:news>
   <news:publication>
    <news:name>
     De Morgen
    </news:name>


In [15]:
links = []
all_links = soup.find_all("loc")
for link in all_links:
    links.append(link.text)
links

['https://www.demorgen.be/snelnieuws/chinese-zoo-in-de-verdediging-hun-honingberen-zijn-echt-en-geen-personeel-in-een-berenpak~b84dab72/',
 'https://www.demorgen.be/meningen/zak-cartoon-van-de-dag~b0de26ebb/',
 'https://www.demorgen.be/politiek/extreemrechts-is-terug-in-griekenland-de-wedergeboorte-van-de-verboden-gouden-dageraad~b3f93966/',
 'https://www.demorgen.be/nieuws/toenemende-spanning-aan-oostgrens-polen-na-incident-met-militaire-helikopters-uit-wit-rusland~b85ec923/',
 'https://www.demorgen.be/oorlog-in-oekraine/live-rusland-bombardeert-oekraiense-graaninfrastructuur-bij-navo-grens-poetin-en-erdogan-hebben-telefonisch-onderhoud~b38bed0a/',
 'https://www.demorgen.be/nieuws/korpschef-van-brussel-zuid-het-grootste-internationale-treinstation-van-belgie-heeft-geen-politiepost-kunt-u-zich-dat-voorstellen~bfcc9790/',
 'https://www.demorgen.be/nieuws/wat-hier-gebeurt-is-ongekend-in-de-geschiedenis-over-de-derde-en-ernstige-strafzaak-tegen-trump~bb89d104/',
 'https://www.demorgen.be/

In [16]:
print(links)


['https://www.demorgen.be/snelnieuws/chinese-zoo-in-de-verdediging-hun-honingberen-zijn-echt-en-geen-personeel-in-een-berenpak~b84dab72/', 'https://www.demorgen.be/meningen/zak-cartoon-van-de-dag~b0de26ebb/', 'https://www.demorgen.be/politiek/extreemrechts-is-terug-in-griekenland-de-wedergeboorte-van-de-verboden-gouden-dageraad~b3f93966/', 'https://www.demorgen.be/nieuws/toenemende-spanning-aan-oostgrens-polen-na-incident-met-militaire-helikopters-uit-wit-rusland~b85ec923/', 'https://www.demorgen.be/oorlog-in-oekraine/live-rusland-bombardeert-oekraiense-graaninfrastructuur-bij-navo-grens-poetin-en-erdogan-hebben-telefonisch-onderhoud~b38bed0a/', 'https://www.demorgen.be/nieuws/korpschef-van-brussel-zuid-het-grootste-internationale-treinstation-van-belgie-heeft-geen-politiepost-kunt-u-zich-dat-voorstellen~bfcc9790/', 'https://www.demorgen.be/nieuws/wat-hier-gebeurt-is-ongekend-in-de-geschiedenis-over-de-derde-en-ernstige-strafzaak-tegen-trump~bb89d104/', 'https://www.demorgen.be/snelnie

149


In [79]:
for link in links:
    link_divs = requests.get(link).content
    soup_link = BeautifulSoup(link_divs, "html.parser")
    link_img = soup_link.find_all('img')
    print(len(link_img))
    print(link)

22
https://www.demorgen.be/snelnieuws/elektrificatie-wagenpark-leidt-tot-langere-interventies-bij-vab-elektrische-wagens-hebben-soms-zelfs-vaker-pech~b80d3089/
22
https://www.demorgen.be/ons-geld/nu-al-voor-een-vast-energiecontract-kiezen-is-misschien-nog-wat-te-vroeg~b07cefcc/
25
https://www.demorgen.be/nieuws/mijn-moeder-belt-me-soms-ongerust-als-ze-een-nummer-van-luis-oscar-heeft-gehoord-zeg-tine-moet-ik-mij-zorgen-maken~b584e453/
49
https://www.demorgen.be/oorlog-in-oekraine/live-oekraine-ook-onkruid-en-struikgroei-hindert-oekraiens-tegenoffensief-zelensky-dringt-aan-op-extra-internationale-hulp-voor-luchtverdediging~b38bed0a/
29
https://www.demorgen.be/niet-te-missen/hoe-kapot-mijn-hart-ook-was-als-mens-ben-ik-voller-geworden-deze-12-interviews-moet-u-gelezen-hebben~beaf4321/
23
https://www.demorgen.be/snelnieuws/ondanks-drie-strafzaken-groeit-voorsprong-donald-trump-op-concurrenten-nog~bb829d1a/
108
https://www.demorgen.be/snelnieuws/ons-kende-ons~b89e9d06/
22
https://www.demorge

In [17]:
special_link = 'https://www.demorgen.be/meningen/van-9-tot-5~bbd332cb/'

link_divs = requests.get(special_link).content
soup_link = BeautifulSoup(link_divs, "html.parser")
link_img = soup_link.find_all('span', {"class": "artstyle__labels__label"})
# for span in link_img:
#     print(span.text)
#print(soup_link.prettify())
print(link_img[0].text)

Cartoon


In [47]:

script = soup_link.find('script', {"type": "application/ld+json"})
data = json.loads(script.text, strict=False)
#print(data['@graph'])
# for serial in data['@graph']['author']:
#     print (serial)
print(data['@graph'][1]['datePublished'])
print(link.text)
time = data['@graph'][1]['datePublished']
print(time)

2023-07-31T15:12:42+0200
https://www.demorgen.be/snelnieuws/brandend-vrachtschip-fremantle-highway-aangekomen-op-tijdelijke-locatie-boven-waddeneilanden~b01622b3/
2023-07-31T15:12:42+0200


In [20]:
link_divs = requests.get(link.text).content
soup_link = BeautifulSoup(link_divs, "html.parser")
# print(soup_link.prettify())
print(link.text)

https://www.demorgen.be/snelnieuws/brandend-vrachtschip-fremantle-highway-aangekomen-op-tijdelijke-locatie-boven-waddeneilanden~b01622b3/


In [21]:
p_tags = soup_link.find_all('p')
p_tags

[<p class="share-modal__copy"> <input class="share-modal__copy-input fjs-share-modal-copy-input" type="hidden" value="https://www.demorgen.be/cs-b01622b3"/> <a class="share-modal__copy-link fjs-share-modal-copy-link" data-gtm="share / top / copylink" href="https://www.demorgen.be/cs-b01622b3"> <svg class="share-modal__copy-icon" height="10" width="20"> <path d="M1.9 5c0-1.7 1.4-3.1 3.1-3.1h4V0H5a5 5 0 0 0 0 10h4V8.1H5A3.1 3.1 0 0 1 1.9 5zM6 6h8V4H6v2zm9-6h-4v1.9h4a3.1 3.1 0 0 1 0 6.2h-4V10h4a5 5 0 0 0 0-10z" fill="#000" fill-rule="nonzero"></path> </svg> <span class="share-modal-copy-link-text fjs-share-modal-copy-link-text" data-click-value="Link gekopieerd"> Kopieer de link van het artikel </span> </a> </p>,
 <p class="artstyle__intro" data-element-id="b296-e010-5153-c1f7-b975-7044-0e3c-dd50">Het brandende vrachtschip Fremantle Highway is aangekomen op een tijdelijke locatie 16 kilometer boven de Nederlandse Waddeneilanden Schiermonnikoog en Ameland. Het verslepen van het brandende v

In [49]:
p_title = soup_link.find_all('title')
Title = unidecode(p_title[0].text.split('|')[0].strip())

Title

'Brandend vrachtschip Fremantle Highway aangekomen op tijdelijke locatie boven Waddeneilanden'

In [41]:
paragraphs = ''
p_intro = soup_link.find_all('p', {"class": "artstyle__intro"})
print(unidecode(p_intro[0].text))
paragraphs  += unidecode(p_intro[0].text.strip())

Het brandende vrachtschip Fremantle Highway is aangekomen op een tijdelijke locatie 16 kilometer boven de Nederlandse Waddeneilanden Schiermonnikoog en Ameland. Het verslepen van het brandende vrachtschip is probleemloos verlopen, zegt de Nederlandse Rijkswaterstaat.
  


In [42]:
p_tags = soup_link.find_all('p', {"class": "artstyle__paragraph"})
p_tags

[<p class="artstyle__paragraph" data-element-id="97b4-1a97-d091-5e9d-de40-07e8-9a13-f842">
       Het verslepen van het vaartuig is gisterenmiddag van start gegaan. Twee sleepboten verplaatsten de Fremantle over 66 kilometer. Rond 11.30 uur maandagmiddag bereikten de vaartuigen de tijdelijke locatie.
     </p>,
 <p class="artstyle__paragraph" data-element-id="abb2-317a-f205-1119-fa78-2b66-932b-88d9">
       “Door de meewerkende stroming tijdens het tweede deel van de sleep kon er sneller gevaren worden dan eerder was berekend”, klinkt het. “Ook de rookontwikkeling van het vrachtschip bleef tijdens de hele sleeproute minimaal.”
     </p>,
 <p class="artstyle__paragraph" data-element-id="9eaa-1d83-f745-7c8b-5795-502c-3cf8-9b98">
       Bedoeling is om het schip te ankeren. Eens dat mogelijk is, zal een team van de bergers aan boord gaan voor een inspectie. Het vrachtschip blijft vastgekoppeld aan de sleepboten en ook een oliebestrijdingsvaartuig blijft in de buurt, in het geval van een o

In [45]:
for parag in p_tags:
    paragraphs += unidecode(parag.text.strip())
print(paragraphs)

Het brandende vrachtschip Fremantle Highway is aangekomen op een tijdelijke locatie 16 kilometer boven de Nederlandse Waddeneilanden Schiermonnikoog en Ameland. Het verslepen van het brandende vrachtschip is probleemloos verlopen, zegt de Nederlandse Rijkswaterstaat.Het verslepen van het vaartuig is gisterenmiddag van start gegaan. Twee sleepboten verplaatsten de Fremantle over 66 kilometer. Rond 11.30 uur maandagmiddag bereikten de vaartuigen de tijdelijke locatie."Door de meewerkende stroming tijdens het tweede deel van de sleep kon er sneller gevaren worden dan eerder was berekend", klinkt het. "Ook de rookontwikkeling van het vrachtschip bleef tijdens de hele sleeproute minimaal."Bedoeling is om het schip te ankeren. Eens dat mogelijk is, zal een team van de bergers aan boord gaan voor een inspectie. Het vrachtschip blijft vastgekoppeld aan de sleepboten en ook een oliebestrijdingsvaartuig blijft in de buurt, in het geval van een olieverontreiniging.De uiteindelijke haven is nog ni

Het brandende vrachtschip Fremantle Highway is aangekomen op een tijdelijke locatie 16 kilometer boven de Nederlandse Waddeneilanden Schiermonnikoog en Ameland. Het verslepen van het brandende vrachtschip is probleemloos verlopen, zegt de Nederlandse Rijkswaterstaat.Het verslepen van het vaartuig is gisterenmiddag van start gegaan. Twee sleepboten verplaatsten de Fremantle over 66 kilometer. Rond 11.30 uur maandagmiddag bereikten de vaartuigen de tijdelijke locatie."Door de meewerkende stroming tijdens het tweede deel van de sleep kon er sneller gevaren worden dan eerder was berekend", klinkt het. "Ook de rookontwikkeling van het vrachtschip bleef tijdens de hele sleeproute minimaal."Bedoeling is om het schip te ankeren. Eens dat mogelijk is, zal een team van de bergers aan boord gaan voor een inspectie. Het vrachtschip blijft vastgekoppeld aan de sleepboten en ook een oliebestrijdingsvaartuig blijft in de buurt, in het geval van een olieverontreiniging.De uiteindelijke haven is nog ni

In [44]:
from unidecode import unidecode
x = unidecode(x)
x

'"Door de meewerkende stroming tijdens het tweede deel van de sleep kon er sneller gevaren worden dan eerder was berekend", klinkt het. "Ook de rookontwikkeling van het vrachtschip bleef tijdens de hele sleeproute minimaal."'

In [261]:
data['url-parent'][2]

'https://www.demorgen.be/snelnieuws/zweden-scherpt-grenscontroles-aan-na-koranverbrandingen~bd4dfe35/'

In [52]:
def get_links(feeds_url):
    page_divs = requests.get(feeds_url).content
    soup = BeautifulSoup(page_divs, "html.parser")
    links = []
    all_links = soup.find_all("loc")
    for link in all_links:
        links.append(link.text)
    return links
    
feeds_url = 'https://www.demorgen.be/sitemaps/news.xml'
links  = get_links(feeds_url)

'https://www.demorgen.be/nieuws/ondanks-drie-strafzaken-groeit-voorsprong-donald-trump-op-concurrenten-nog~bb829d1a/'

In [13]:
import itertools
from bs4 import BeautifulSoup
from concurrent.futures import ThreadPoolExecutor
from tqdm.contrib.concurrent import thread_map
from functools import partial

/home/mahmoud/Desktop/Training/Data-Tank-project/env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [136]:
def get_title(soup): # Extract title 
    return soup.find_all('title')[0].text.split('|')[0].strip()

def get_text(soup): # Extract the article's paragraphs
    paragraphs = ''
    p_intro = soup.find_all('p', {"class": "artstyle__intro"}) # article intro
    if p_intro:
        paragraphs  += p_intro[0].text.strip()
    p_tags = soup.find_all('p', {"class": "artstyle__paragraph"})
    for parag in p_tags:
        paragraphs += parag.text.strip() # adding article's paragraphs
    return paragraphs

def get_date(soup): # Extract the published date
    script = soup.find('script', {"type": "application/ld+json"})
    data = json.loads(script.text, strict=False)
    return data['@graph'][1]['datePublished']

def get_links(feeds_url): # Extract links from news feed
    page_divs = requests.get(feeds_url).content
    soup = BeautifulSoup(page_divs, "html.parser")
    links = []
    all_links = soup.find_all("loc")
    for link in all_links:
        links.append(link.text)
    return links
feeds_url = 'https://www.demorgen.be/sitemaps/news.xml'

links  = get_links(feeds_url)
source_url = 'https://www.demorgen.be/tv-cultuur/euphoria-acteur-angus-cloud-25-overleden~b94ad30d/'
session = requests.Session()
page_html = session.get(source_url).content
soup = BeautifulSoup(page_html, "html.parser")
page_type = soup.find_all('span', {"class": "artstyle__labels__label"})
if (page_type[0].text == 'Cartoon'):
            print('Cartoon page ! ')
else:
     article_title = get_title(soup)
     article_text = get_text(soup)
     published_date = get_date(soup)
     row = [[source_url,article_title,article_text,published_date,'NL']]
     df = pd.DataFrame(row,columns = ["Source_url", "article_title", "article_text","published_date","language"])
     #dnew_row = {"Source_url":source_url, "article_title":article_title, "article_text":article_text,"published_date":published_date,"language":'NL'}
     #df.loc[len(df)] = new_row

/home/mahmoud/Desktop/Training/Data-Tank-project/env/lib/python3.10/site-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


In [139]:
df

,Source_url,article_title,article_text,published_date,language
0,https://www.demorgen.be/tv-cultuur/euphoria-ac...,‘Euphoria’-acteur Angus Cloud (25) overleden,"Acteur Angus Cloud, vooral bekend van de HBO-s...",2023-08-02T03:00:03+0200,NL


In [140]:
def get_title(soup): # Extract title 
    return soup.find_all('title')[0].text.split('|')[0].strip()

def get_text(soup): # Extract the article's paragraphs
    paragraphs = ''
    p_intro = soup.find_all('p', {"class": "artstyle__intro"}) # article intro
    if p_intro:
        paragraphs  += p_intro[0].text.strip()
    p_tags = soup.find_all('p', {"class": "artstyle__paragraph"})
    for parag in p_tags:
        paragraphs += parag.text.strip() # adding article's paragraphs
    return paragraphs

def get_date(soup): # Extract the published date
    script = soup.find('script', {"type": "application/ld+json"})
    data = json.loads(script.text, strict=False)
    return data['@graph'][1]['datePublished']
    
def get_links(feeds_url): # Extract links from news feed
    page_divs = requests.get(feeds_url).content
    soup = BeautifulSoup(page_divs, "html.parser")
    links = []
    all_links = soup.find_all("loc")
    for link in all_links:
        links.append(link.text)
    return links

def get_article(url, session):

    try:
        page_html = session.get(url).content
        soup = BeautifulSoup(page_html, "html.parser")
        page_type = soup.find_all('span', {"class": "artstyle__labels__label"})

        if (page_type[0].text == 'Cartoon'):
            return None
        else:
            article_title = get_title(soup)
            article_text = get_text(soup)
            published_date = get_date(soup)
            row = [[url,article_title,article_text,published_date,'NL']]
            df = pd.DataFrame(row,columns = ["Source_url", "article_title", "article_text","published_date","language"])
        
        return df
    
    except Exception as e:
        print(type(e))
        return e

if __name__ == "__main__":

    feeds_url = 'https://www.demorgen.be/sitemaps/news.xml'
    links  = get_links(feeds_url)

    with requests.Session() as session:
        news_feed = pd.concat(df for df in thread_map(partial(get_article, session=session), links) 
                               if isinstance(df, pd.DataFrame))
    
    
    
    news_feed.to_csv("Demorgen.csv")






/home/mahmoud/Desktop/Training/Data-Tank-project/env/lib/python3.10/site-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(
  3%|▎         | 4/148 [00:02<01:01,  2.36it/s]

<class 'IndexError'>


100%|██████████| 148/148 [00:23<00:00,  6.28it/s]


In [141]:
news_feed

,Source_url,article_title,article_text,published_date,language
0,https://www.demorgen.be/snelnieuws/lizzo-ontke...,Lizzo ontkent beschuldigingen van wangedrag: ‘...,Lizzo ontkent dat er sprake is geweest van sek...,2023-08-03T13:43:00+0200,NL
0,https://www.demorgen.be/nieuws/adidas-kan-verl...,Adidas kan verlies beperken door sneakers van ...,Adidas heeft sinds juni 400 miljoen euro omzet...,2023-08-03T13:00:00+0200,NL
0,https://www.demorgen.be/nieuws/parket-start-on...,Parket start onderzoek naar vader van Francis ...,Het parket van Oost-Vlaanderen heeft een onder...,2023-08-03T12:17:02+0200,NL
0,https://www.demorgen.be/snelnieuws/biden-vraag...,Biden vraagt ‘onmiddellijke vrijlating’ van af...,De Amerikaanse president Joe Biden heeft donde...,2023-08-03T12:10:00+0200,NL
0,https://www.demorgen.be/oorlog-in-oekraine/gee...,"Geen kindergeld, rijbewijs of medische zorgen:...",Oekraïense staatsburgers in de door Rusland be...,2023-08-03T11:28:50+0200,NL
...,...,...,...,...,...
0,https://www.demorgen.be/snelnieuws/zweden-sche...,Zweden scherpt grenscontroles aan na koranverb...,De Zweedse regering gaat controles aan de gren...,2023-08-01T14:36:50+0200,NL
0,https://www.demorgen.be/nieuws/sekstoerisme-in...,Sekstoerisme in het Franse Cap d’Agde: ‘Als ee...,"In de Franse oksel van de Middellandse Zee, op...",2023-08-01T14:00:02+0200,NL
0,https://www.demorgen.be/nieuws/neurowetenschap...,Neurowetenschapper over spierziekte ALS: ‘Tast...,Presentatrice en zangeres Bea Van der Maat (62...,2023-08-01T14:00:01+0200,NL
0,https://www.demorgen.be/oorlog-in-oekraine/vie...,Vierde droneaanval op Moskou in een week: ‘Dit...,In Moskou is vannacht opnieuw een gebouw aange...,2023-08-01T13:57:36+0200,NL


In [134]:
for x in links:
    print(x)

https://www.demorgen.be/oorlog-in-oekraine/live-oekraine-ook-onkruid-en-struikgroei-hindert-oekraiens-tegenoffensief-kathedraal-in-cherson-beschadigd-bij-russisch-bombardement~b38bed0a/
https://www.demorgen.be/oorlog-in-oekraine/hoe-oekrainers-in-bezette-gebieden-massaal-gedwongen-worden-russisch-paspoort-aan-te-nemen-klassieke-vorm-van-oorlogsmisdaad~bbaa17754/
https://www.demorgen.be/nieuws/ok-boomer-op-tiktok-wemelt-het-van-negatieve-stereotypen-van-ouderen~b65a62fc/
https://www.demorgen.be/snelnieuws/elektrificatie-wagenpark-leidt-tot-langere-interventies-bij-vab-elektrische-wagens-hebben-soms-zelfs-vaker-pech~b80d3089/
https://www.demorgen.be/ons-geld/nu-al-voor-een-vast-energiecontract-kiezen-is-misschien-nog-wat-te-vroeg~b07cefcc/
https://www.demorgen.be/nieuws/kinderen-van-succesvolle-ouders-zouden-hun-connecties-gebruiken-om-hogerop-te-raken-het-lot-van-nepobaby-s-zeker~b584e453/
https://www.demorgen.be/niet-te-missen/blijkbaar-maak-ik-sommige-mensen-boos-door-over-mijn-ziekte

In [135]:
temp_links = links
for x in news_feed['Source_url']:
    if x in temp_links:
        temp_links.remove(x)
for yyy in temp_links:
    print(yyy)

https://www.demorgen.be/oorlog-in-oekraine/live-oekraine-ook-onkruid-en-struikgroei-hindert-oekraiens-tegenoffensief-kathedraal-in-cherson-beschadigd-bij-russisch-bombardement~b38bed0a/
https://www.demorgen.be/snelnieuws/ons-kende-ons~b89e9d06/
https://www.demorgen.be/meningen/zak-cartoon-van-de-dag~b0de26ebb/
https://www.demorgen.be/meningen/van-9-tot-5~bbd332cb/
https://www.demorgen.be/meningen/een-rib-uit-het-lijf~b650230e/
